In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from imutils import paths
import numpy as np
import argparse
import cv2
import os
import glob
import pymc3 as pm



import theano 
import theano.tensor as T

Using TensorFlow backend.


In [2]:
def image_to_feature_vector(image, size=(32, 32)): return cv2.resize(image, size).flatten()

In [3]:
cats = map(image_to_feature_vector, [cv2.imread(cat) for cat in glob.glob('data/cats/*')])
dogs = map(image_to_feature_vector, [cv2.imread(dog) for dog in glob.glob('data/dogs/*')])

data = np.vstack((cats,dogs))
labels = ['cat']*len(cats) + ['dog']*len(dogs)

In [4]:
le = LabelEncoder()
labels = le.fit_transform(labels)
data = np.array(data) / 255.0
labels = np_utils.to_categorical(labels, 2)

In [5]:
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [6]:
model = Sequential()
model.add(Dense(768, input_dim=3072, init="uniform",
	activation="relu"))
model.add(Dense(384, init="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(768, activation="relu", kernel_initializer="uniform", input_dim=3072)`
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(384, activation="relu", kernel_initializer="uniform")`


In [7]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
model.fit(trainData, trainLabels, nb_epoch=50, batch_size=128, verbose=1)

[INFO] compiling model...


/usr/local/lib/python2.7/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
300/300 [==============================] - 0s - loss: 0.7064 - acc: 0.5000     
Epoch 2/50
300/300 [==============================] - 0s - loss: 0.7019 - acc: 0.5033     
Epoch 3/50
300/300 [==============================] - 0s - loss: 0.6840 - acc: 0.5433     
Epoch 4/50
300/300 [==============================] - 0s - loss: 0.6718 - acc: 0.5933     
Epoc

In [8]:
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy*100))


[INFO] evaluating on testing set...
100/100 [==============================] - 0s
[INFO] loss=0.6830, accuracy: 59.0000%


In [9]:
X, Y = data, labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5)
ann_input = theano.shared(data)
ann_output = theano.shared(labels)

# Initialize random weights between each layer
init_1 = np.random.randn(X.shape[1], 768)
init_2 = np.random.randn(768, 384)
init_out = np.random.randn(384)

In [10]:
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    weights_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                             shape=(X.shape[1], 768), 
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    weights_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                            shape=(768, 384), 
                            testval=init_2)
    
    # Weights from hidden lay2er to output
    weights_2_out = pm.Normal('w_2_out', 0, sd=1, 
                              shape=(384,), 
                              testval=init_out)
    
    # Build neural-network using tanh activation function
    act_1 = T.nnet.relu(T.dot(ann_input, 
                         weights_in_1))
    act_2 = T.nnet.relu(T.dot(act_1, 
                         weights_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, 
                                   weights_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', 
                       act_out,

                       observed=ann_output)
"""
model = Sequential()
model.add(Dense(768, input_dim=3072, init="uniform",
	activation="relu"))
model.add(Dense(384, init="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
"""    

/usr/local/lib/python2.7/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


ValueError: Input dimension mis-match. (input[0].shape[1] = 2, input[1].shape[1] = 400)